# ------------------------------------------------------------------------------------------------------------------------------------------PyMiiam - Que vais-je manger ce soir ?-------------------------------------------------------------------------------------------------------------------------------------------------

PyMiiam est une application qui permet à un utilisateur de trouver des restaurants en fournissant sa localisation et le type de repas qu'il cherche

Développeurs: 
- Abdoul Yacine Oumarou Faroukou
- Maman Mouzamil Mahaman
- Jonathan Ray

## Installation de librairies et modules

Certains des modules dont nous aurons besoin ne sont pas installés de base avec Anaconda. Il est dès lors important de s'assurer de rouler cette cellule pour que tout le programme fonctionne correctement.

In [1]:
#Ce module nous permettra de faire des graphiques de type cartes et d'y ajouter des points
!pip install ipyleaflet

#À cette étape on ajoute le module installé précedemment dans les registres de Jupyter Notebook,
#Sinon le programme n'affichera pas les cartes. Il est important que ces cellules roulent sans erreurs
#pour pouvoir un affichage de la carte
!jupyter nbextension enable --py widgetsnbextension --sys-prefix
!jupyter nbextension enable --py --sys-prefix ipyleaflet

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: ok
Enabling notebook extension jupyter-leaflet/extension...
      - Validating: ok


## Importation de toutes les librairies et modules dont nous aurons besoin

In [1]:
import requests #Module permettant d'envoyer des requêtes serveur
import json #Module permettant de lire des fichiers au format json
import pandas as pd
from ipyleaflet import Map, basemaps, Marker #Module permettant de faire des cartes interactives
# Le module email nous permettra de gérer et d'envoyer des courriels
from email.mime.text import MIMEText
from email.mime.application import MIMEApplication
from email.mime.multipart import MIMEMultipart
# le module smtplib permettra de créer des sessions par lesquelles nous enverrons nos emails
from smtplib import SMTP
import smtplib as smtplib
import datetime 
import time

## Création de la variable dont nous aurons besoin pour l'ensemble du programme

In [2]:
#On intialise ici un objet dataframe qui contiendra l'essentiel de l'information
#concernant les restaurants
restaurants = pd.DataFrame()    

## Création des fonctions que le programme va utiliser pour fonctionner

- Fonction pour rechercher les restaurants

In [3]:
# Notre fonction prendra deux arguments : le lieu de la recherche ainsi que le mot clé
# associé à la recherche
def business_finder(location, keyword):
    
    # On initialise d'abord la variable url, qui est le lien vers lequel on fera les recherches
    # agit comme un moteur de recherche Google par exemple
    url = "https://api.yelp.com/v3/businesses/search"
    
    # Premièrement, on ouvre le fichier qui continer notre API key et
    # on enregistre cette clé dans la variable key. L'API key est indispensable
    # pour que le serveur puisse nous reconnaître
    with open('api_key.txt') as file:
        key = file.read() #on lit le fichier ici
        file.close() #ensuite on ferme le fichier pour sauver de la mémoire

    # on défini les paramètres d'autorisation afin que Yelp puisse nous identifier
    # et qu'on puisse recevoir des données. Cette étape est indispensable aussi
    # car elle permet à la plateforme de nous identifier grâce à l'api key
    headers = {
      "Authorization": "Bearer " + key
    }
    
    # Dans cette section, on entre tous les paramètres dont le client veux effectuer
    # sa recherche à savoir ou il se trouve et le mot clé de la recherche. À noter
    # que keyword et location sont des arguments de la fonction. Nous avons aussi un 
    # élément categories qu'on a fixé à restaurants car on veux chercher uniquement les
    # restaurants
    params = {
        "term": keyword,
        "location": location,
        "categories": "restaurants"
    }
    
    # On envoie une requête à l'API avec nos paramètres grâce à request et on enregistre
    # le résultat dans un variable response. Cette variable est en réalisté un fichier json
    # et donc pour faciliter sa lecture on le préfixe avec la méthode json() en prenant soint
    # de juste garder 'businesses' qui représente tous les résultats de la recherche. Ces résultats
    # sont enregistrés dans une nouvelle variable businesses
    response = requests.get(url, headers= headers, params= params)
    businesses = response.json()['businesses']

    # On initialise ici des listes vides qui vont venir accueillir les attributs dont aura besoin
    # pour remplir notre dataframe
    name = []
    coordinates = []
    latitude = []
    longitude = []
    phone = []
    location = []
    
    # Grâce à une boucle on va parcourir la variable businesses qui est en réalité un dictionnaire
    for business in businesses:
        name.append(business['name']) # on ajoute le nom à name
        coordinates.append(business['coordinates']) # on ajoute les coordonées à coordinates
        phone.append(business['phone']) # on ajoute le téléphone à phone
        location.append(business['location']['address1'] + ", " + business['location']['city'] 
                        + ", " + business['location']['state'] + ", " + business['location']['zip_code'])
                        # on ajoute l'adresse en concaténant plusieurs valeurs à location
        
    # la liste coordinate contient en réalité des dictionnaires, on va créer une boucle pour en 
    # extraire les valeurs
    for i in range(len(coordinates)):
        latitude.append(coordinates[i]['latitude']) # on ajoute la latitude à latitude
        longitude.append(coordinates[i]['longitude'])# on ajoute la longitude à longitude
    
    # Finalement, on rempli notre dataframe restaurants avec toutes les données contenues dans les listes
    restaurants['Nom'] = name
    restaurants['Phone Number'] = phone
    restaurants['Adresse'] = location
    restaurants['Latitude'] = latitude
    restaurants['Longitude'] = longitude
    
    # Et pour finir, on demande à la fonction de nous retourner les 5 premiers résultats (Nom du restaurant ainsi
    # que son numéro de téléphone)
    return restaurants[['Nom', 'Phone Number']].head()
    

- Fonction pour faire la carte et y afficher les restaurants

In [4]:
# Notre fonction pour créer la carte prendre uniquement le dataframe restaurants comme arguments
def map_plot(dataframe):
    
    # D'abord on s'assure de retenir les 5 premiers résultats
    dataframe = dataframe.head()
    
    # Ici on créer tout d'abord la carte dont le centre sera la moyenne des latitudes et des longitudes
    # de nos restaurants, on place également le zoom à 12 pour avoir tous les détails
    montreal = Map(center= (dataframe['Latitude'].mean(), dataframe['Longitude'].mean()), zoom= 12)
    
    # On va ensuite créer les marqueurs des restaurants grâce à leur coordonnées. Le titre de chaque 
    # marqueur sera en réalité le nom du restaurant suivi de son adresse postale
    for (index, row) in dataframe.iterrows():
        marker = Marker(location = [row.loc['Latitude'], row.loc['Longitude']], title= row.loc['Nom'] + ", " +
                        row.loc['Adresse'])
        # On ajoute ici chaque marqueur à la carte
        montreal.add_layer(marker)
    
    # Finalement on demande à la fonction de nous retrouner la carte finale
    return montreal

- Fonction pour envoyer un email à l'utilisateur

In [5]:
# Cette fonction prend deux arguments, l'adresse mail sur laquelle on va envoyer le message
# ainsi que notre dataframe restaurants
def send_email(email, dataframe):
    
    # D'abord on s'assure de retenir les 5 premiers résultats
    dataframe = dataframe[['Nom', 'Phone Number', 'Adresse']].head()
    
    # On créer une instance de la classe MIMEMultipart qui sera le corps de notre message
    # cette classe comprend plusieurs attributs
    message = MIMEMultipart()
    
    # On défini de qui vient le message d'abord
    message['from'] = 'PyMiiam'
    
    # Ensuite à qui l'on veux envoyer le message (1er argument de la fonction)
    message['to'] = email
    
    # Enusite le titre du message
    message['subject'] = 'Your search result on {0}'.format(datetime.date.today())
    
    # Finalement le contenu du message qui sera en fait du code HTML auquel on a
    # ajouté notre dataframe
    html = """\
        <html>
          <head></head>
          <body>
            {0}
          </body>
        </html>
        """.format(dataframe.to_html())
    
    # On ajoute le contenu au message
    message.attach(MIMEText(html, 'html'))
    
    # On va maintenant contacter le serveur qui va envoyer le message. On utilise le
    # service de google
    with smtplib.SMTP(host= 'smtp.gmail.com', port= 587) as smtp:
        # On fait hello au serveur pour qu'il sache qu'on est là
        smtp.ehlo()
        # Ensuite on lui fait savoir qu'on veux envoyer un email
        smtp.starttls()
        # On entre nos identifiant ici. À noter que pour des raisons de sécurités,
        # il serait préférable de garder ses informations dans un fichier séparé,
        # mais pour alleger le programme nous l'avons intégré directement
        smtp.login("py.pymiiam@gmail.com", "pymiiam1234")
        # On envoi le message
        smtp.send_message(message)
        # Et on affiche un message comme quoi l'email a bien été envoyé
        print('Email envoyé à : {0}'.format(email))


## L'application

In [6]:
# On s'assure tout d'abord de bien réinitialiser le dataframe dans le cas ou le client
# voudrait faire plusieurs recherches
restaurants = pd.DataFrame()  

# Un peu d'espace
print('\n')

# On salue l'utilisateur et on lui explique le principe de l'application
print("Bonjour et bienvenue dans PyMiiam ! Une application qui vous permet de trouver des restaurants prêts de chez-vous !")
# On attend 2 secondes pour rendre le programme plus fluide
time.sleep(2)
# Un peu d'espace
print('\n')

# On demande ici à l'utilisateur sa localisation et on l'enregistre dans la variable location
location = input('Veuillez tout d''abord nous indiquer dans quelle ville vous souhaitez manger. \n\
Par exemple : Montréal, mtl, montreal. ')
print('Parfait !')
# 1 seconde d'attente
time.sleep(1)
# Un peu d'espace
print('\n')

# On demande ici à l'utilisateur ce qu'il veux manger
keyword = input('Veuillez nous indiquer maintenant ce que vous aimeriez manger. \n\
Par exemple : sushis, poulet, riz, steak. ')
print('Parfait ! Nous procédons à la recherche...')
# 2 secondes d'attente
time.sleep(2)
# Un peu d'espace
print('\n')

#On fait recherche en appelant la fonction business_finder
print('Voici les meilleurs résultats que nous avons trouvé : ')
resultats = business_finder(location= location, keyword= keyword) 
print('\n')

# On affiche les résultats de la recherche
print('------------------------------------------------------------------------------') # améliorer la lisibilité
print(resultats)
print('------------------------------------------------------------------------------') # améliorer la lisibilité

# Un peu d'espace
print('\n')

# On va maintenant demander à l'utilisateur s'il veux recevoir un email des résultats.
# On va valider la réponse grâce à une boucle
while True:
    try:
        # On s'attend à 1 ou 2 comme réponse
        response = int(input('Voulez-vous recevoir un email des résultats ? \n 1 - Oui \n 2 - Non \n Votre réponse : '))
    except Exception:
        # S'il entre un string à la place d'un int cela génère une erreur. On va alors
        # lui signaler
        print("Veuillez entrez 1 ou 2 s'il vous plaît")
        continue #la boucle continue de tourner
    # Si la réponse est un int mais différent de 1 ou 2 on va lui signaler encore
    if (response != 1 and response != 2):
        print("Veuillez entrez 1 ou 2 s'il vous plaît")
        continue #la boucle continue
    # Sinon la réponse est 1 ou 2. On sort de la boucle
    else:
        #sinon, on sort de la boucle ici (donc la valeur entrée est 1 ou 2)
        break
        

# Un peu d'espace
print('\n')

# On va également valider l'adresse mail grâce à une boucle. On enverra un mail seulement
# si l'utilisateur à entrer 1 précédemment. C'est donc notre condition de départ
if response == 1:
    # On initialise la boucle
    while True:
        try:
            # On lui demande son adresse mail et si elle est bonne 
            # on envoi l'email en appelant la fonction
            email = input("Veuillez entrer votre adresse email : ")
            send_email(email= email, dataframe= restaurants)
            break #on sort de la boucle si sans erreur
        # En cas d'erreur (donc adresse invalide), on lui signale et on continue la boucle
        except Exception:
            print("Veuillez entrez une adresse mail valide s'il vous plait")
            continue

# Un peu d'espace
print('\n')

# Finalement on appelle notre fonction map_plot pour afficher les résultats
map_plot(restaurants)



Bonjour et bienvenue dans PyMiiam ! Une application qui vous permet de trouver des restaurants prêts de chez-vous !


Veuillez tout dabord nous indiquer dans quelle ville vous souhaitez manger. 
Par exemple : Montréal, mtl, montreal. 3742, CHEMIN STE-FOY
Parfait !


Veuillez nous indiquer maintenant ce que vous aimeriez manger. 
Par exemple : sushis, poulet, riz, steak. riz
Parfait ! Nous procédons à la recherche...


Voici les meilleurs résultats que nous avons trouvé : 


------------------------------------------------------------------------------
           Nom  Phone Number
0    Chez Tao!  +14182041850
1        Calao  +14186534999
2  Le Chavigny  +14182864959
------------------------------------------------------------------------------


Voulez-vous recevoir un email des résultats ? 
 1 - Oui 
 2 - Non 
 Votre réponse : 2






Map(center=[46.7394746512706, -71.5061988189936], controls=(ZoomControl(options=['position', 'zoom_in_text', '…